In [1]:
import pandas as pd
import numpy as np
from titlecase import titlecase
import requests
import datetime
import json
from humps.camel import case

In [2]:
# CSV inlezen
df = pd.read_csv('https://maps.amsterdam.nl/open_geodata/excel.php?KAARTLAAG=FUNCTIEKAART&THEMA=functiekaart', delimiter=';')

In [6]:
df.Hoofdfunctie.value_counts()

Detailhandel             10501
Kantoren                  8053
Horeca                    3975
Bedrijven                 2924
Opslag en distributie     1207
Atelier / werkruimte      1185
Onderwijs                 1114
Onduidelijke functie       896
Zorg                       804
Logies                     716
Sport en recreatie         637
Cultuur                    472
Religie                    220
Wijk / buurtcentrum        122
Nuts                        96
Agrarisch bedrijf           71
_Niet laten zien             2
Name: Hoofdfunctie, dtype: int64

In [8]:
df.WOZ_gebruik.value_counts()

winkel                                      9081
kantoor                                     5411
café / bar / restaurant                     3195
overig bedrijf                              2934
kantoor in bedrijfsverzamelgebouw           1418
                                            ... 
congresgebouw                                  1
kapel                                          1
tuinbouwbedrijf (incl bloembollenteelt)        1
(poli)kliniek                                  1
woning + intensieve veehouderij pluimvee       1
Name: WOZ_gebruik, Length: 120, dtype: int64

In [5]:
# Deze data pipeline geldt niet meer, nieuwe functiekaart heeft heel andere indeling

# Converteer LAT LNG columns naar een punt als decimal
df['lon'] = df['LNG']
df['lat'] = df['LAT']

# Categorieën die we gaan uitsluiten (waardes van de FUNCTIE1_OMS kolom)
nietrelevant_FUNCTIE1_OMS = 'Bedrijven|Kantoren|Openbaar vervoer|Parkeren'

# Subcategorien die we gaan uitsluiten (waardes van de FUNCTIE2_OMS kolom)
nietrelevant_FUNCTIE2_OMS = 'Opstappunt rondvaart - waterfiets|Onduidelijk'

# Kolommen die we willen behouden
# 'ADRESSEN_LIJST'
relevante_kolommen = ['FUNCTIE1_OMS', 'FUNCTIE2_OMS', 'ZAAKNAAM', 'lon', 'lat', 'ZAAK_ID']

# Nieuwe dataframe zonder niet relevante data
df_filtered = df[relevante_kolommen]
df_filtered = df_filtered[~df_filtered['FUNCTIE1_OMS'].str.contains(nietrelevant_FUNCTIE1_OMS, na=False)]
df_filtered = df_filtered[~df_filtered['FUNCTIE2_OMS'].str.contains(nietrelevant_FUNCTIE2_OMS, na=False)]
df_filtered = df_filtered.fillna('')

# Hernoem detailhandel naar winkel, horeca naar overige horeca
# We houden cafés en coffeeshops apart van horeca
df_filtered['cat'] = df_filtered['FUNCTIE1_OMS']
df_filtered['cat'] = df_filtered['cat'].replace({'Detailhandel':'Winkels', 'Horeca': 'Horeca - Overig'})
df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Café - Eetcafé', 'cat'] = "Cafés"
df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Coffeeshop', 'cat'] = "Coffeeshops"

# Zelfde voor uitgaan: gokken, bioscopen en bordelen eruit
df_filtered['cat'] = df_filtered['cat'].replace({'Uitgaan en Toerisme': 'Uitgaan en Toerisme - Overig'})
df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Automatenhal - Casino', 'cat'] = "Gokhal"
df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Bioscoop - Filmhuis', 'cat'] = "Bioscoop"
df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Bordeel - Raamprostitutie - Sexclub', 'cat'] = "Prostitutie"

# Zelfde voor zorg: dag- en nachtopvang eruit
df_filtered['cat'] = df_filtered['cat'].replace({'Zorg': 'Zorg - Overig'})
df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Dag- en nachtopvang', 'cat'] = "Dag- en nachtopvang"

# Output filtered vs hele dataset, telling en output waarden eerste keer draaien script
print('Waarde 2018-07-20:')
print('18900 objecten (filtered) vs 28189 (hele dataset)')
print('Waarde ' + str(datetime.datetime.today()).split()[0] + ':')
print(str(len(df_filtered)) + " objecten (filtered) vs " + str(len(df)) + " (hele dataset)")

# Titlecase names with a locally adjusted version of the titlecase 
# module: https://github.com/ppannuto/python-titlecase
from titlecase import titlecase

def title_exceptions(word, **kwargs):
#     print(word)
    if word.upper() in ('BV', 'AKO','AH','PT020', 'IJ', 'VTP', 'ID', 'TV', 'HTML', 'XML', "URL", "HTTP", "VPN"):
        return word.upper()
    if word.upper() in ('VOF'):
        return word.lower()
    if "IJ" in word[:2]:
        return word.lower().replace('ij', 'IJ')

df_filtered['name'] = df_filtered['ZAAKNAAM'].apply((lambda x: titlecase(x, callback=title_exceptions)))

# Only keep what we need, rename columns
df_final = df_filtered[['name', 'cat', 'lon', 'lat', 'ZAAK_ID']].copy()
df_final.columns = ['name', 'cat', 'lon', 'lat', 'zaak_id']

# Write to JSON as records
df_final.to_json('../src/assets/data/functies.json', orient="records")

# Also create category JSON (todo: escape cafés to cafes)
with open('../src/assets/data/functies-categories.json', 'w') as outfile:
    json.dump([ {'category':x, 'camelName': case(x)} for x in df_final['cat'].unique()], outfile)



KeyError: "['ZAAK_ID', 'FUNCTIE2_OMS', 'FUNCTIE1_OMS', 'ZAAKNAAM'] not in index"

In [3]:
# File setup has changed to saner defaults, see next cell for current cleanup

# # Converteer LAT LNG columns naar een punt als decimal
# df['lon'] = pd.to_numeric(df['LNG'].str.replace(',','.'))
# df['lat'] = pd.to_numeric(df['LAT'].str.replace(',','.'))

# # Categorieën die we gaan uitsluiten (waardes van de FUNCTIE1_OMS kolom)
# nietrelevant_FUNCTIE1_OMS = 'Bedrijven|Kantoren|Openbaar vervoer|Parkeren'

# # Subcategorien die we gaan uitsluiten (waardes van de FUNCTIE2_OMS kolom)
# nietrelevant_FUNCTIE2_OMS = 'Opstappunt rondvaart - waterfiets|Onduidelijk'

# # Kolommen die we willen behouden
# # 'ADRESSEN_LIJST'
# relevante_kolommen = ['FUNCTIE1_OMS', 'FUNCTIE2_OMS', 'ZAAKNAAM', 'lon', 'lat', 'ZAAK_ID']

# # Nieuwe dataframe zonder niet relevante data
# df_filtered = df[relevante_kolommen]
# df_filtered = df_filtered[~df_filtered['FUNCTIE1_OMS'].str.contains(nietrelevant_FUNCTIE1_OMS, na=False)]
# df_filtered = df_filtered[~df_filtered['FUNCTIE2_OMS'].str.contains(nietrelevant_FUNCTIE2_OMS, na=False)]
# df_filtered = df_filtered.fillna('')

# # Hernoem detailhandel naar winkel, horeca naar overige horeca
# # We houden cafés en coffeeshops apart van horeca
# df_filtered['cat'] = df_filtered['FUNCTIE1_OMS']
# df_filtered['cat'] = df_filtered['cat'].replace({'Detailhandel':'Winkels', 'Horeca': 'Horeca - Overig'})
# df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Café - Eetcafé', 'cat'] = "Cafés"
# df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Coffeeshop', 'cat'] = "Coffeeshops"

# # Zelfde voor uitgaan: gokken, bioscopen en bordelen eruit
# df_filtered['cat'] = df_filtered['cat'].replace({'Uitgaan en Toerisme': 'Uitgaan en Toerisme - Overig'})
# df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Automatenhal - Casino', 'cat'] = "Gokhal"
# df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Bioscoop - Filmhuis', 'cat'] = "Bioscoop"
# df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Bordeel - Raamprostitutie - Sexclub', 'cat'] = "Prostitutie"

# # Zelfde voor zorg: dag- en nachtopvang eruit
# df_filtered['cat'] = df_filtered['cat'].replace({'Zorg': 'Zorg - Overig'})
# df_filtered.loc[df_filtered['FUNCTIE2_OMS'] == 'Dag- en nachtopvang', 'cat'] = "Dag- en nachtopvang"

# # Output filtered vs hele dataset, telling en output waarden eerste keer draaien script
# print('Waarde 2018-07-20:')
# print('18900 objecten (filtered) vs 28189 (hele dataset)')
# print('Waarde ' + str(datetime.datetime.today()).split()[0] + ':')
# print(str(len(df_filtered)) + " objecten (filtered) vs " + str(len(df)) + " (hele dataset)")

# # Titlecase names with a locally adjusted version of the titlecase 
# # module: https://github.com/ppannuto/python-titlecase
# from titlecase import titlecase

# def title_exceptions(word, **kwargs):
# #     print(word)
#     if word.upper() in ('BV', 'AKO','AH','PT020', 'IJ', 'VTP', 'ID', 'TV', 'HTML', 'XML', "URL", "HTTP", "VPN"):
#         return word.upper()
#     if word.upper() in ('VOF'):
#         return word.lower()
#     if "IJ" in word[:2]:
#         return word.lower().replace('ij', 'IJ')

# df_filtered['name'] = df_filtered['ZAAKNAAM'].apply((lambda x: titlecase(x, callback=title_exceptions)))

# # Only keep what we need, rename columns
# df_final = df_filtered[['name', 'cat', 'lon', 'lat', 'ZAAK_ID']].copy()
# df_final.columns = ['name', 'cat', 'lon', 'lat', 'zaak_id']

# # Write to JSON as records
# df_final.to_json('../src/assets/data/functies.json', orient="records")

# # Also create category JSON (todo: escape cafés to cafes)
# with open('../src/assets/data/functies-categories.json', 'w') as outfile:
#     json.dump([ {'category':x, 'camelName': case(x)} for x in df_final['cat'].unique()], outfile)



AttributeError: Can only use .str accessor with string values!

In [4]:
[ {'category':x, 'camelName': case(x)} for x in df_final['cat'].unique() ]

[{'camelName': 'winkels', 'category': 'Winkels'},
 {'camelName': 'zorgOverig', 'category': 'Zorg - Overig'},
 {'camelName': 'activiteitenEnOntmoeting',
  'category': 'Activiteiten en Ontmoeting'},
 {'camelName': 'horecaOverig', 'category': 'Horeca - Overig'},
 {'camelName': 'onderwijs', 'category': 'Onderwijs'},
 {'camelName': 'uitgaanEnToerismeOverig',
  'category': 'Uitgaan en Toerisme - Overig'},
 {'camelName': 'dagEnNachtopvang', 'category': 'Dag- en nachtopvang'},
 {'camelName': 'religie', 'category': 'Religie'},
 {'camelName': 'sport', 'category': 'Sport'},
 {'camelName': 'cafés', 'category': 'Cafés'},
 {'camelName': 'gokhal', 'category': 'Gokhal'},
 {'camelName': 'coffeeshops', 'category': 'Coffeeshops'},
 {'camelName': 'prostitutie', 'category': 'Prostitutie'},
 {'camelName': 'bioscoop', 'category': 'Bioscoop'}]

In [4]:
def get_access_token(login_endpoint, username, password):
    r = requests.post(login_endpoint, {"username": username, "password": password})
    return r.json()['access_token']

In [5]:
login_endpoint = 'https://api.stadswerken.amsterdam/login'
endpoint = 'https://api.stadswerken.amsterdam/address_research_data'
token = get_access_token(login_endpoint, 'jurian', 'forest kosher thalamus shipyard hereby')
headers = {"Authorization":"Bearer %s" %token}

In [6]:
def list_records(endpoint, headers):
    r = requests.get(endpoint, headers=headers)
    return r.json()

In [7]:
def post_new_record(endpoint, jsondata, headers):
    requests.post(endpoint, json=jsondata, headers=headers)

In [8]:
# import time
# for row in df_final.to_dict(orient="records"):
#     post_new_record(endpoint, row, headers)

In [13]:
def list_all_records(filter_string):
    skip = 0
    allrecords = []
    recordsfound = True
    while recordsfound:
        records_on_page = list_records(endpoint + "?$skip=" + str(skip) + filter_string, headers)
        if (records_on_page != []):
            allrecords.append(records_on_page)
            skip += len(records_on_page)
        else:
            recordsfound = False
    return [item for sublist in allrecords for item in sublist]

In [14]:
def get_array_of_api_values(column_name, fstring):
    return [d[column_name] for d in list_all_records(filter_string=fstring)]

In [15]:
get_array_of_api_values('name', '')

['Kalender Kleermaker',
 'Willems Vintage Bikes',
 'Gezichtsreflexologie',
 'Eva',
 'Kas Keerweer',
 'Irenes Hobby Hoek',
 'De IJstuin',
 'Mili Goros',
 'Las Reisbureau',
 'Geveke Werktuigbouw',
 'Mango Mobility',
 'Van Ree Groothandel Dieren Benodigdheden',
 'Horeca Groothandel en Fietswinkel',
 'Ducati Winkel en Service',
 'Opleidingscentrum Westpoort Veiligheid en Mobiliteit',
 'De Diamant Zalenverhuur',
 'Dentacare',
 'Woon Thuis in de Stad',
 'Dr Chicken',
 'De Visser Dental B.V.',
 'Lieve Schoot',
 'Thuiszorg de Lindeboom',
 'Mix Kitchen',
 'Stern/Arend Autos',
 'Praxis',
 'Van Kouwen Amsterdam',
 'The Hand Clinic',
 'Shri Sitaram Dhaam',
 'BUBLé STUDY CENTER',
 'Afura',
 'PT020',
 'Tula Yogastudio',
 'Fysiotherapie',
 '',
 'Kledingrepratie',
 'Stichting Kumbet',
 'Maria Jobse',
 'Integraal Medisch Centrum',
 'Tandarts',
 'Baram Sanitair en Tegels',
 'Boomerang Zorg',
 'Accuraat Begeleid Wonen',
 'Stomerij',
 'De Medische Voet',
 'Make Over Factory',
 'Psyq',
 'Dis Studio Boffi',

In [16]:
len(get_array_of_api_values('id', ''))

17875

In [18]:
def update_record(endpoint_id, jsondata, headers):
    requests.put(endpoint_id, json=jsondata, headers=headers)

In [19]:
# empty = {}

# for id in [d['id'] for d in list_all_records()]:
#     update_record(endpoint + '/' + id, empty, headers)

In [ ]:
list_all_records()

In [21]:
len(list_all_records(filter_string="&$filter=id eq 'a6e05e8f-a507-4262-96d4-0315f8b89ab1'"))

0

In [22]:
df_final["cat"].unique()

array(['Winkels', 'Zorg - Overig', 'Activiteiten en Ontmoeting',
       'Horeca - Overig', 'Onderwijs', 'Uitgaan en Toerisme - Overig',
       'Dag- en nachtopvang', 'Religie', 'Sport', 'Cafés', 'Gokhal',
       'Coffeeshops', 'Prostitutie', 'Bioscoop'], dtype=object)

In [30]:
def list_column_value_duplicates(column_value, all_records):
    all_records_var = all_records
    column_value_set = set(d.get(column_value) for d in all_records_var)
    new_dict = {}
    duplicates = []
    for k in column_value_set:
        records_with_column_value = [d for d in all_records_var if d.get(column_value) == k]
        new_dict[k] = records_with_column_value
    for row in new_dict:
        if (len(new_dict[row]) > 1):
            duplicates.append(new_dict[row])
    duplicate_ids = []
    for dupe in duplicates:
        duplicate_ids.append(dupe[1:][0]['id'])
    return duplicate_ids

In [31]:
bla = list_column_value_duplicates('zaak_id', list_all_records(filter_string=""))
bla

['ec0af4cc-05ce-474d-9b62-bd985b523e16',
 '8b36d123-54ac-4a7a-9c0c-552dad3ed457',
 '72af5326-10df-4288-b905-4e5c68047e62',
 '7e3b04e8-3fab-4eec-94fa-78787690014c',
 '66ed51bf-1e3e-42a7-8514-0f5863bef553',
 '70a4d08d-d35a-45d2-b854-77fd2e3450c9',
 '4c4a00c2-cb4a-4acf-a467-49d97fd8575a',
 '2eb335be-467e-4c0c-a4f6-753808d069fe',
 '2f92ed0e-cb37-49c4-afe5-fa17aada66a9',
 '276ef496-0830-4819-b4b1-20ab83b556f3',
 '0003c029-78ec-42bd-bffd-abd48a42ef52',
 'c6830456-e7de-4a4c-8d2e-ed6403a7a54f',
 '27f1ec16-93ef-4640-9463-d089989d016f',
 'faea7cf0-ac6a-4e6e-81c5-7910ac988e23',
 'ed75cbd3-150c-4219-8dd5-21b67c309653',
 '0dc54b91-b38d-483d-a120-1bf99400c02b',
 '82a9f296-2649-45e4-ac43-ad1f52d688ea',
 'f95eef91-517f-4fa6-8b2f-f2d1adb3d70f',
 'a57aadec-0875-442d-9d17-284c1487f893',
 'f95adcae-a9d6-4a3a-940d-97ccac71a736',
 '0559a2ef-1b19-4dc8-bb2a-57489c1cef13',
 'a21640c9-1aed-4d12-a834-f67b58bf3d2d',
 '132ceb53-4f3b-4518-85bf-27cab70b0e36',
 '6ea56d22-e5bd-45ad-bc74-724cd1dc2c8f',
 '1b4c32be-78b2-

In [20]:
# empty = {}

# for id in bla:
#     update_record(endpoint + '/' + id, empty, headers)

In [32]:
len(bla)

17375

In [24]:
len(list_all_records(filter_string="&$filter=cat gt 0"))

68500

In [25]:
winkels = list_all_records(filter_string="&$filter=cat eq 'Winkels'")
onderwijs = list_all_records(filter_string="&$filter=cat eq 'Onderwijs'")

In [26]:
len(winkels)

31928

In [23]:
df_final.groupby(['cat']).size()

cat
Activiteiten en Ontmoeting       639
Bioscoop                          13
Cafés                           1308
Coffeeshops                      188
Dag- en nachtopvang               67
Gokhal                            24
Horeca - Overig                 3279
Onderwijs                       1096
Prostitutie                       96
Religie                          289
Sport                            524
Uitgaan en Toerisme - Overig     176
Winkels                         8732
Zorg - Overig                   1444
dtype: int64

In [24]:
list_all_records(filter_string="")

[{'cat': 'Winkels',
  'id': '99ea4ffe-6353-4613-9307-8bd21ca6fd87',
  'lat': 52.368449,
  'lon': 4.8676639999999995,
  'name': 'Kalender Kleermaker',
  'zaak_id': 4915.0},
 {'cat': 'Winkels',
  'id': '2d684161-005c-4fb4-9307-5aade89316e2',
  'lat': 52.372624,
  'lon': 4.8594915,
  'name': 'Willems Vintage Bikes',
  'zaak_id': 5995.0},
 {'cat': 'Zorg - Overig',
  'id': '0e5b2031-d3a9-4a79-bab8-8f622f16cf1d',
  'lat': 52.3644325,
  'lon': 4.8567815,
  'name': 'Gezichtsreflexologie',
  'zaak_id': 6398.0},
 {'cat': 'Winkels',
  'id': '86f0d059-f03c-4ede-ad17-5ee6de1535e8',
  'lat': 52.364543,
  'lon': 4.8773985,
  'name': 'Eva',
  'zaak_id': 6533.0},
 {'cat': 'Activiteiten en Ontmoeting',
  'id': '24942ab6-94c6-4887-997f-2922ee04ef26',
  'lat': 52.370221,
  'lon': 4.8639135,
  'name': 'Kas Keerweer',
  'zaak_id': 6542.0},
 {'cat': 'Winkels',
  'id': '99e37a6b-db8b-4392-abe5-0d9437d7017c',
  'lat': 52.3832005,
  'lon': 4.8720935,
  'name': 'Irenes Hobby Hoek',
  'zaak_id': 8912.0},
 {'cat':

In [42]:
list_all_records(filter_string="&$filter=cat eq 'Zorg - Overig'")

[]

In [26]:
list_all_records(filter_string="&$filter=cat eq ''")

[{'cat': 'Winkels',
  'id': '99ea4ffe-6353-4613-9307-8bd21ca6fd87',
  'lat': 52.368449,
  'lon': 4.8676639999999995,
  'name': 'Kalender Kleermaker',
  'zaak_id': 4915.0},
 {'cat': 'Winkels',
  'id': '2d684161-005c-4fb4-9307-5aade89316e2',
  'lat': 52.372624,
  'lon': 4.8594915,
  'name': 'Willems Vintage Bikes',
  'zaak_id': 5995.0},
 {'cat': 'Zorg - Overig',
  'id': '0e5b2031-d3a9-4a79-bab8-8f622f16cf1d',
  'lat': 52.3644325,
  'lon': 4.8567815,
  'name': 'Gezichtsreflexologie',
  'zaak_id': 6398.0},
 {'cat': 'Winkels',
  'id': '86f0d059-f03c-4ede-ad17-5ee6de1535e8',
  'lat': 52.364543,
  'lon': 4.8773985,
  'name': 'Eva',
  'zaak_id': 6533.0},
 {'cat': 'Activiteiten en Ontmoeting',
  'id': '24942ab6-94c6-4887-997f-2922ee04ef26',
  'lat': 52.370221,
  'lon': 4.8639135,
  'name': 'Kas Keerweer',
  'zaak_id': 6542.0},
 {'cat': 'Winkels',
  'id': '99e37a6b-db8b-4392-abe5-0d9437d7017c',
  'lat': 52.3832005,
  'lon': 4.8720935,
  'name': 'Irenes Hobby Hoek',
  'zaak_id': 8912.0},
 {'cat':

In [28]:
empty_values = get_array_of_api_values('id', "&$filter=cat eq ''")

In [29]:
len(empty_values)

68500

In [29]:
def delete_record(endpoint_id, headers):
    requests.delete(endpoint_id, headers=headers)

In [30]:
# for v in get_array_of_api_values('id', "&$filter=cat eq ''"):
#     delete_record(endpoint + '/' + v, headers)

In [31]:
list_all_records(filter_string="")

[{'cat': 'Winkels',
  'id': '99ea4ffe-6353-4613-9307-8bd21ca6fd87',
  'lat': 52.368449,
  'lon': 4.8676639999999995,
  'name': 'Kalender Kleermaker',
  'zaak_id': 4915.0},
 {'cat': 'Winkels',
  'id': '2d684161-005c-4fb4-9307-5aade89316e2',
  'lat': 52.372624,
  'lon': 4.8594915,
  'name': 'Willems Vintage Bikes',
  'zaak_id': 5995.0},
 {'cat': 'Zorg - Overig',
  'id': '0e5b2031-d3a9-4a79-bab8-8f622f16cf1d',
  'lat': 52.3644325,
  'lon': 4.8567815,
  'name': 'Gezichtsreflexologie',
  'zaak_id': 6398.0},
 {'cat': 'Winkels',
  'id': '86f0d059-f03c-4ede-ad17-5ee6de1535e8',
  'lat': 52.364543,
  'lon': 4.8773985,
  'name': 'Eva',
  'zaak_id': 6533.0},
 {'cat': 'Activiteiten en Ontmoeting',
  'id': '24942ab6-94c6-4887-997f-2922ee04ef26',
  'lat': 52.370221,
  'lon': 4.8639135,
  'name': 'Kas Keerweer',
  'zaak_id': 6542.0},
 {'cat': 'Winkels',
  'id': '99e37a6b-db8b-4392-abe5-0d9437d7017c',
  'lat': 52.3832005,
  'lon': 4.8720935,
  'name': 'Irenes Hobby Hoek',
  'zaak_id': 8912.0},
 {'cat':

[![Screen_Shot_2018-07-31_at_14.54.34.png](https://s8.postimg.cc/zan7875xh/Screen_Shot_2018-07-31_at_14.54.34.png)](https://postimg.cc/image/yl4evu5dt/)